In [3]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import numpy as np
import codecs
from pathlib import Path

url = 'https://understat.com/match/11919'
url1 = 'https://understat.com/league/EPL'

p = requests.get(url1)
pdecoded = codecs.decode(p.text,'unicode_escape')
page = BeautifulSoup(p.text, 'html.parser')
a = []
for tags in page('script'):
    if '= JSON.parse' in str(tags):
        for els in str(tags).split():
            if 'JSON' in els:
                els = els[12:-3]
                els = codecs.decode(els,'unicode_escape')
                a.append(json.loads(els))
TT = pd.DataFrame(a[1]).transpose()
UnderstatTeams = dict(zip(TT['id'], TT['title']))
Schedule = pd.DataFrame(a[0])



Table = pd.DataFrame()
i=0
while Schedule.at[i,'isResult']:
    url = 'https://understat.com/match/'+ Schedule.at[i,'id']
    #print(url)
    p = requests.get(url)
    pdecoded = codecs.decode(p.text,'unicode_escape')
    page = BeautifulSoup(p.text, 'html.parser')
    tempList = []
    for tags in page('script'):
        if '= JSON.parse' in str(tags):
            for els in str(tags).split():
                if 'JSON' in els:
                    els = els[12:-3]
                    els = codecs.decode(els,'unicode_escape')
                    tempList.append(json.loads(els))
    Table = Table.append(pd.DataFrame(tempList[2]['a']).transpose(), ignore_index=True)
    Table = Table.append(pd.DataFrame(tempList[2]['h']).transpose(), ignore_index=True)
    i+=1
Table['team'] = [UnderstatTeams[Table.at[i,'team_id']] for i in Table.index]
#Table = pd.DataFrame(a[2]['h']).transpose().append(pd.DataFrame(a[2]['a']).transpose(), ignore_index=True)
#Table
Table.to_csv(Path('out/Understat.csv'))
Table

,id,goals,own_goals,shots,xG,time,player_id,team_id,position,player,...,red_card,roster_in,roster_out,key_passes,assists,xA,xGChain,xGBuildup,positionOrder,team
0,341166,0,0,0,0,90,982,79,GK,Tim Krul,...,0,0,0,0,0,0,0.054997265338897705,0.054997265338897705,1,Norwich
1,341167,0,0,0,0,90,7688,79,DR,Max Aarons,...,0,0,0,0,0,0,0.4084494411945343,0.4084494411945343,2,Norwich
2,341168,0,0,1,0.008998467586934566,90,7689,79,DC,Ben Godfrey,...,0,0,0,0,0,0,0.06399573385715485,0.054997265338897705,3,Norwich
3,341169,0,1,0,0,90,7690,79,DC,Grant Hanley,...,0,0,0,0,0,0,0,0,3,Norwich
4,341170,0,0,0,0,90,7691,79,DL,Jamal Lewis,...,0,0,0,2,0,0.10757879167795181,0.2609913647174835,0.16241103410720825,4,Norwich
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7459,387437,0,0,6,0.651094913482666,68,6049,220,AML,Solly March,...,0,387441,0,1,0,0.02469896525144577,0.6502277255058289,0,13,Brighton
7460,387438,0,0,5,0.5699573755264282,90,3621,220,FW,Neal Maupay,...,0,0,0,0,0,0,1.1717329025268555,0.6618022322654724,15,Brighton
7461,387439,0,0,0,0,6,8020,220,Sub,Steven Alzate,...,0,0,387433,0,0,0,0,0,17,Brighton
7462,387440,0,0,2,0.06645529717206955,10,6842,220,Sub,Alireza Jahanbakhsh,...,0,0,387429,0,0,0,0.05149408429861069,0,17,Brighton


In [41]:
TT = pd.DataFrame(a[1]).transpose()
UnderstatTeams = dict(zip(TT['id'], TT['title']))
UnderstatTeams


{'71': 'Aston Villa',
 '72': 'Everton',
 '73': 'Bournemouth',
 '74': 'Southampton',
 '75': 'Leicester',
 '78': 'Crystal Palace',
 '79': 'Norwich',
 '80': 'Chelsea',
 '81': 'West Ham',
 '82': 'Tottenham',
 '83': 'Arsenal',
 '86': 'Newcastle United',
 '87': 'Liverpool',
 '88': 'Manchester City',
 '89': 'Manchester United',
 '90': 'Watford',
 '92': 'Burnley',
 '220': 'Brighton',
 '229': 'Wolverhampton Wanderers',
 '238': 'Sheffield United'}

In [2]:
Table.keys()

Index(['id', 'goals', 'own_goals', 'shots', 'xG', 'time', 'player_id',
       'team_id', 'position', 'player', 'h_a', 'yellow_card', 'red_card',
       'roster_in', 'roster_out', 'key_passes', 'assists', 'xA', 'xGChain',
       'xGBuildup', 'positionOrder'],
      dtype='object')

In [52]:
Schedule#.at[0,'h']

,id,isResult,h,a,goals,xG,datetime,forecast
0,11643,True,"{'id': '87', 'title': 'Liverpool', 'short_titl...","{'id': '79', 'title': 'Norwich', 'short_title'...","{'h': '4', 'a': '1'}","{'h': '2.23456', 'a': '0.842407'}",2019-08-09 20:00:00,"{'w': '0.7377', 'd': '0.1732', 'l': '0.0891'}"
1,11644,True,"{'id': '81', 'title': 'West Ham', 'short_title...","{'id': '88', 'title': 'Manchester City', 'shor...","{'h': '0', 'a': '5'}","{'h': '1.2003', 'a': '3.18377'}",2019-08-10 12:30:00,"{'w': '0.0452', 'd': '0.1166', 'l': '0.8382'}"
2,11645,True,"{'id': '73', 'title': 'Bournemouth', 'short_ti...","{'id': '238', 'title': 'Sheffield United', 'sh...","{'h': '1', 'a': '1'}","{'h': '1.34099', 'a': '1.59864'}",2019-08-10 15:00:00,"{'w': '0.2083', 'd': '0.3923', 'l': '0.3994'}"
3,11646,True,"{'id': '92', 'title': 'Burnley', 'short_title'...","{'id': '74', 'title': 'Southampton', 'short_ti...","{'h': '3', 'a': '0'}","{'h': '0.909241', 'a': '1.08752'}",2019-08-10 15:00:00,"{'w': '0.266', 'd': '0.3442', 'l': '0.3898'}"
4,11647,True,"{'id': '78', 'title': 'Crystal Palace', 'short...","{'id': '72', 'title': 'Everton', 'short_title'...","{'h': '0', 'a': '0'}","{'h': '0.87159', 'a': '1.2246'}",2019-08-10 15:00:00,"{'w': '0.2427', 'd': '0.3231', 'l': '0.4342'}"
...,...,...,...,...,...,...,...,...
375,12018,False,"{'id': '75', 'title': 'Leicester', 'short_titl...","{'id': '89', 'title': 'Manchester United', 'sh...","{'h': None, 'a': None}","{'h': None, 'a': None}",2020-05-17 14:00:00,NaN
376,12019,False,"{'id': '72', 'title': 'Everton', 'short_title'...","{'id': '73', 'title': 'Bournemouth', 'short_ti...","{'h': None, 'a': None}","{'h': None, 'a': None}",2020-05-17 14:00:00,NaN
377,12020,False,"{'id': '78', 'title': 'Crystal Palace', 'short...","{'id': '82', 'title': 'Tottenham', 'short_titl...","{'h': None, 'a': None}","{'h': None, 'a': None}",2020-05-17 14:00:00,NaN
378,12021,False,"{'id': '80', 'title': 'Chelsea', 'short_title'...","{'id': '229', 'title': 'Wolverhampton Wanderer...","{'h': None, 'a': None}","{'h': None, 'a': None}",2020-05-17 14:00:00,NaN


In [173]:
pd.DataFrame(a[2]['a']).transpose()

,id,goals,own_goals,shots,xG,time,player_id,team_id,position,player,...,yellow_card,red_card,roster_in,roster_out,key_passes,assists,xA,xGChain,xGBuildup,positionOrder
382610,382610,0,0,0,0,90,745,75,GK,Kasper Schmeichel,...,1,0,0,0,0,0,0,0,0,1
382611,382611,0,0,0,0,90,3303,75,DR,Ricardo Pereira,...,0,0,0,0,1,0,0.11073379963636398,0.32855525612831116,0.21782146394252777,2
382612,382612,0,0,1,0.0843578353524208,90,5264,75,DC,Caglar Söyüncü,...,0,0,0,0,0,0,0,0.08558996021747589,0.08558996021747589,3
382613,382613,0,0,1,0.01846957765519619,90,807,75,DC,Jonny Evans,...,0,0,0,0,0,0,0,0.029888764023780823,0.029888764023780823,3
382614,382614,0,0,2,0.03182113915681839,90,782,75,DL,Ben Chilwell,...,0,0,0,0,2,0,0.11098882555961609,0.35096216201782227,0.2502477765083313,4
382615,382615,0,0,1,0.062379028648138046,68,6418,75,DMC,Hamza Choudhury,...,0,0,382622,0,0,0,0,0.13914074003696442,0.13914074003696442,7
382616,382616,0,0,4,0.40932855010032654,79,770,75,MR,Ayoze Pérez,...,0,0,382623,0,1,0,0.14593414962291718,0.6540447473526001,0.098782017827034,8
382617,382617,0,0,2,0.1406225711107254,74,1234,75,MC,Dennis Praet,...,1,0,382621,0,0,0,0,0.3326757252216339,0.1920531541109085,9
382618,382618,0,0,0,0,90,6818,75,MC,James Maddison,...,0,0,0,0,4,0,0.14377854764461517,0.2710821330547333,0.2710821330547333,9
382619,382619,0,0,4,0.16415247321128845,90,6681,75,ML,Harvey Barnes,...,0,0,0,0,4,0,0.32178622484207153,0.48077383637428284,0.18271812796592712,10


In [136]:
ss = codecs.decode(a[0],'unicode_escape')

In [137]:
pd.DataFrame(ss)

ValueError: DataFrame constructor not properly called!